In [1]:
import SVMTK as svmtk
from pathlib import Path

In [2]:
def create_ventricle_free_mesh(brain_stl, ventricle_stl, output_mesh, resolution):
    # Create svmtk surfaces
    brain = svmtk.Surface(brain_stl)
    ventricles = svmtk.Surface(ventricle_stl)
    
    smap = svmtk.SubdomainMap()
    smap.add("10", 1)
    smap.add("01", 2)
    
    domain = svmtk.Domain([brain, ventricles], smap)
    domain.create_mesh(resolution)
    
    domain.remove_subdomain(2)
    
    print("Saving mesh at ", Path(output_mesh).resolve())
    domain.save(str(Path(output_mesh).resolve()))
    

def svmtk_stl_to_mesh(infile_stl, outfile_mesh, resolution):
    all_surf = svmtk.Surface(infile_stl)

    domain = svmtk.Domain(all_surf)
    domain.create_mesh(resolution)
    domain.save(outfile_mesh)

In [3]:
brainstl = "../mridata/stls/brainventriclemerge_Brain.stl"
ventriclestl = "../mridata/stls/brainventriclemerge_Ventricles.stl"
output_file_no_ventricles = "../mridata/svmtk-mesh-files/ratbrain-no-ventricles.mesh"
output_file = "../mridata/svmtk-mesh-files/ratbrain.mesh"

res = 16
create_ventricle_free_mesh(brainstl, ventriclestl, output_file_no_ventricles, resolution=res)
svmtk_stl_to_mesh(brainstl, output_file, resolution=res)


0 holes have been filled
Cell size: 1.25823
Start meshing
Number of isolated vertices removed: 25
Done meshing
Number of isolated vertices removed: 28
Number of removed subdomain cells : 4279
Saving mesh at  /home/jorgen/ratbrain/mridata/svmtk-mesh-files/ratbrain-no-ventricles.mesh

0 holes have been filled
Cell size: 1.25823
Start meshing
Number of isolated vertices removed: 22
Done meshing


1. Create .mesh-file both using the `create_ventricle_free_mesh`-function, and the , and `smtk_stl_to_mesh`, with resolution 16.
1. Create one xdmfdir per meshfile.
1. Run `convert_to_dolfin_mesh` on both meshfiles with the corresponding svmtk-meshes.
1. Visualize the two meshes side by side in paraview.

Comparison of the two functions with resolution 16. (Left) Without ventricles subdomain removal. (Right) with ventricle removal. 

![Comparison 1](../visualization/svmtk-remove-ventricles-comparison1.png)
![Comparison 2](../visualization/svmtk-remove-ventricles-comparison2.png)

## Conclusion
There does not seem to be an obvious advantage of using one approach over the other. It seems the ventricle removal-approach preserves more of the ventricles, but both cases suffer of some asymmetry-problems, where the ventricles in one side of the brain are preserved, but not in the other. 

### Scratchpad